In [5]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from scipy import stats
import numpy as np
import h5py
import pandas as pd
import tensorflow as tf

In [6]:
f = h5py.File('C:/Users/huboz/Desktop/Pjatk/HARDataset/full_preprocesed.h5', 'r')

In [7]:
print(list(f.keys()))

['x', 'y_features', 'y_move', 'y_person']


In [15]:

x = f['x'][:,:,-16:]
y = f['y_move']

In [16]:
print(x.shape,y.shape)

(2927, 425, 16) (2927, 1)


In [17]:
def load_group(filenames):
    loaded = list()
    for name in filenames:
        data = f
        loaded.append(data)
    # grupę stosów, tak aby funkcje były trzecim wymiarem
    loaded = dstack(loaded)
    return loaded

In [18]:
import numpy as np

X = x[0:2927]
Y = y[0:2927]
   
   

In [19]:
c = list(zip(X,Y))
np.random.shuffle(c)

In [20]:
def load_dataset():
    
   
    c = list(zip(X,Y))
    np.random.shuffle(c)
    
    TrainX = X[0:2047]
    TestX = X[2048:2927]
    TrainY = Y[0:2047]
    TestY = Y[2048:2927]

    TrainY = to_categorical(TrainY)
    TestY = to_categorical(TestY)
    
    return TrainX, TrainY, TestX, TestY

In [21]:
load_dataset()

(array([[[ 1.30208853e-01, -2.84142152e-03,  3.92287044e-02, ...,
           1.95956089e-02,  4.21961549e-02,  3.24824422e-02],
         [ 1.63553743e-01,  2.81515983e-04,  5.09274047e-02, ...,
           2.65451790e-02,  6.56309042e-02,  5.28208185e-02],
         [ 1.96924450e-01,  3.34332876e-03,  6.25297690e-02, ...,
           3.33250057e-02,  8.85738783e-02,  7.24812301e-02],
         ...,
         [ 3.04264934e-01,  1.73424214e-02,  1.18609637e-01, ...,
           4.72713753e-02,  9.18297540e-02,  1.72180678e-01],
         [ 3.04022134e-01,  1.73334941e-02,  1.18516435e-01, ...,
           4.72960850e-02,  9.18958161e-02,  1.72398558e-01],
         [ 3.03945564e-01,  1.73308746e-02,  1.18488291e-01, ...,
           4.73039420e-02,  9.19162231e-02,  1.72470514e-01]],
 
        [[ 9.07021586e-02, -3.97322117e-03,  1.82756512e-02, ...,
          -1.66066097e-02, -1.17348478e-02,  4.64909426e-02],
         [ 1.44857488e-01,  6.57794046e-04,  4.84356636e-02, ...,
          -1.20554172

In [22]:
def evaluate_model(TrainX, TrainY, TestX, TestY):
    verbose, epochs, batch_size = 1, 10, 64 # 15, 64 domyslnie, dla 10, 64 działa najlepiej verbose pokaze liczbe epok
    
    n_timesteps, n_features, n_outputs = TrainX.shape[1], TrainX.shape[2], TrainY.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.1)) # 0.5
    
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(TrainX, TrainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(TestX, TestY, batch_size=batch_size, verbose=1)
    return accuracy

In [23]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [24]:
def run_experiment(repeats=10):
    # load data
    TrainX, TrainY, TestX, TestY = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(TrainX, TrainY, TestX, TestY)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [25]:
run_experiment(repeats=10)

Epoch 1/10
32/32 [==============================] - 9s 296ms/step - loss: 1.4842 - accuracy: 0.3166
Epoch 2/10
32/32 [==============================] - 9s 289ms/step - loss: 1.2491 - accuracy: 0.4436
Epoch 3/10
32/32 [==============================] - 10s 309ms/step - loss: 1.1688 - accuracy: 0.4880
Epoch 4/10
32/32 [==============================] - 11s 350ms/step - loss: 1.1274 - accuracy: 0.4831
Epoch 5/10
32/32 [==============================] - 11s 349ms/step - loss: 1.1675 - accuracy: 0.4841
Epoch 6/10
32/32 [==============================] - 11s 356ms/step - loss: 1.0810 - accuracy: 0.5247
Epoch 7/10
32/32 [==============================] - 12s 367ms/step - loss: 1.0145 - accuracy: 0.5545
Epoch 8/10
32/32 [==============================] - 12s 379ms/step - loss: 0.9719 - accuracy: 0.5901
Epoch 9/10
32/32 [==============================] - 14s 425ms/step - loss: 1.0192 - accuracy: 0.5823
Epoch 10/10
14/14 [==============================] - 2s 141ms/step - loss: 1.9051 - accuracy:

32/32 [==============================] - 14s 444ms/step - loss: 1.1006 - accuracy: 0.5105
Epoch 6/10
32/32 [==============================] - 14s 446ms/step - loss: 1.0497 - accuracy: 0.5232
Epoch 7/10
32/32 [==============================] - 14s 450ms/step - loss: 1.0359 - accuracy: 0.5540
Epoch 8/10
32/32 [==============================] - 14s 453ms/step - loss: 1.0994 - accuracy: 0.5398
Epoch 9/10
32/32 [==============================] - 14s 450ms/step - loss: 1.1995 - accuracy: 0.5012
Epoch 10/10
14/14 [==============================] - 1s 87ms/step - loss: 1.5721 - accuracy: 0.4653
>#8: 46.530
Epoch 1/10
32/32 [==============================] - 14s 423ms/step - loss: 1.4642 - accuracy: 0.3298
Epoch 2/10
32/32 [==============================] - 14s 426ms/step - loss: 1.2900 - accuracy: 0.4333
Epoch 3/10
32/32 [==============================] - 13s 420ms/step - loss: 1.1850 - accuracy: 0.4856
Epoch 4/10
32/32 [==============================] - 14s 426ms/step - loss: 1.1918 - accurac